In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from random import randint
import random
from time import sleep
from pandas import json_normalize
import pandas as pd
import pickle
import ast
from sklearn.preprocessing import StandardScaler

#### Import 100 hot songs

In [2]:
hot100songs = pd.read_csv('100songs.csv')
hot100songs

,title,artists
0,Flowers,Miley Cyrus
1,Last Night,Morgan Wallen
2,Fast Car,Luke Combs
3,Like Crazy (Deep House Remix),Jimin
4,Like Crazy (UK Garage Remix),Jimin
...,...,...
95,"Going, Going, Gone",Luke Combs
96,Calm Down,Rema
97,Heart Attack (Rock Version),Demi Lovato
98,Tennessee Orange,Megan Moroney


#### Import clustered song collection

In [3]:
clustered_songs = pd.read_csv('clustered_songs.csv')
clustered_songs

,title,artists,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,cluster
0,Take Me To Church,['Hozier'],spotify:track:7dS5EaCoMnN7DzlpT6aRn2,0.566,0.664,4,-5.303,0,0.0464,0.634000,...,0.1160,0.437,128.945,audio_features,7dS5EaCoMnN7DzlpT6aRn2,https://api.spotify.com/v1/tracks/7dS5EaCoMnN7...,https://api.spotify.com/v1/audio-analysis/7dS5...,241688,4,0
1,Cooler Than Me - Single Mix,"['Mike Posner', 'Gigamesh']",spotify:track:2V4bv1fNWfTcyRJKmej6Sj,0.768,0.820,7,-4.630,0,0.0474,0.179000,...,0.6890,0.625,129.965,audio_features,2V4bv1fNWfTcyRJKmej6Sj,https://api.spotify.com/v1/tracks/2V4bv1fNWfTc...,https://api.spotify.com/v1/audio-analysis/2V4b...,213293,4,0
2,See You Again (feat. Kali Uchis),"['Tyler, The Creator', 'Kali Uchis']",spotify:track:7KA4W4McWYRpgf0fWsJZWB,0.558,0.559,6,-9.222,1,0.0959,0.371000,...,0.1090,0.620,78.558,audio_features,7KA4W4McWYRpgf0fWsJZWB,https://api.spotify.com/v1/tracks/7KA4W4McWYRp...,https://api.spotify.com/v1/audio-analysis/7KA4...,180387,4,1
3,Pompeii,['Bastille'],spotify:track:3gbBpTdY8lnQwqxNCcf795,0.679,0.715,9,-6.383,1,0.0407,0.075500,...,0.2710,0.571,127.435,audio_features,3gbBpTdY8lnQwqxNCcf795,https://api.spotify.com/v1/tracks/3gbBpTdY8lnQ...,https://api.spotify.com/v1/audio-analysis/3gbB...,214148,4,2
4,Hips Don't Lie (feat. Wyclef Jean),"['Shakira', 'Wyclef Jean']",spotify:track:3ZFTkvIE7kyPt6Nu3PEa7V,0.778,0.824,10,-5.892,0,0.0707,0.284000,...,0.4050,0.758,100.024,audio_features,3ZFTkvIE7kyPt6Nu3PEa7V,https://api.spotify.com/v1/tracks/3ZFTkvIE7kyP...,https://api.spotify.com/v1/audio-analysis/3ZFT...,218093,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7982,Hot N Cold,['Katy Perry'],spotify:track:1ulXmqjktxSdrdT1bzc69H,0.706,0.841,7,-3.956,1,0.0418,0.000079,...,0.0688,0.861,132.032,audio_features,1ulXmqjktxSdrdT1bzc69H,https://api.spotify.com/v1/tracks/1ulXmqjktxSd...,https://api.spotify.com/v1/audio-analysis/1ulX...,220227,4,2
7983,What Lovers Do,"['Maroon 5', 'SZA']",spotify:track:0ExtPzAWi4arQz6kaabm02,0.799,0.597,5,-5.131,0,0.0611,0.078800,...,0.1000,0.419,110.001,audio_features,0ExtPzAWi4arQz6kaabm02,https://api.spotify.com/v1/tracks/0ExtPzAWi4ar...,https://api.spotify.com/v1/audio-analysis/0Ext...,199849,4,0
7984,You Need To Calm Down - Clean Bandit Remix,"['Taylor Swift', 'Clean Bandit']",spotify:track:7Bjf96uNizpvXLvNyOJpst,0.726,0.880,2,-4.137,1,0.0581,0.072000,...,0.0906,0.739,95.037,audio_features,7Bjf96uNizpvXLvNyOJpst,https://api.spotify.com/v1/tracks/7Bjf96uNizpv...,https://api.spotify.com/v1/audio-analysis/7Bjf...,152773,4,2
7985,Barracuda,['Heart'],spotify:track:4KfSdst7rW39C0sfhArdrz,0.548,0.691,4,-13.553,0,0.0369,0.015000,...,0.1440,0.667,137.145,audio_features,4KfSdst7rW39C0sfhArdrz,https://api.spotify.com/v1/tracks/4KfSdst7rW39...,https://api.spotify.com/v1/audio-analysis/4KfS...,261933,4,0


#### Function from part 2

In [31]:
def SongRecommender1():
    song_input = input("Enter your favorite song: ")
    if song_input in hot100songs['title'].unique():
        random_song = hot100songs['title'][random.randint(0, len(hot100songs)-1)]
        random_artist = hot100songs.loc[hot100songs.title == random_song, 'artists'].values[0]
        return 'Look at this new hot song: '+random_song+' from '+random_artist
    else:
        return 'Sorry, cannot give any recommendation. Try another one'

In [5]:
#Test it
#SongRecommender1()

#### Load kmeans model

In [6]:
kmeans = pickle.load(open('kmeans_model.sav', 'rb'))
print(kmeans)

KMeans(n_clusters=5, random_state=1234)


#### Load Scaler

In [7]:
scaler = pickle.load(open('standard_scaler.sav', 'rb'))
print(scaler)

StandardScaler()


#### Connection to Spotify API

In [8]:
secrets_file = open("C:/Users/lisab/OneDrive/Desktop/DA_labs/Week_6/Spotify_API.txt")

In [9]:
string = secrets_file.read()

In [10]:
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()

In [11]:
#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['Client ID'],
                                                           client_secret=secrets_dict['Client Secret']))

#### Develop else part of SongRecommender Function

In [14]:
song_spotify = sp.search(q='Flowers', limit=1)
song_uri = song_spotify['tracks']['items'][0]['uri']
song_audio_features = sp.audio_features(song_uri)

In [15]:
song_audio_features

In [16]:
audio_feat_test = pd.DataFrame(song_audio_features)
audio_feat_test

In [17]:
audio_feat_df_test = audio_feat_test[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 
                          'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']].copy()
audio_feat_df_test

In [18]:
from sklearn.preprocessing import StandardScaler
audio_feat_df_prep_test = scaler.transform(audio_feat_df_test)

#Now, all features will have the same weight.
pd.DataFrame(audio_feat_df_prep_test,columns=audio_feat_df_test.columns).head()

In [19]:
#Predicting / assigning the clusters:
clusters_test = kmeans.predict(audio_feat_df_prep_test)

In [20]:
clusters_test

In [21]:
clusters_test[0]

In [22]:
#take clustered_songs dataframe and search for a song with the same number as clusters_test
clustered_songs_input_cluster = clustered_songs[clustered_songs['cluster'] == clusters_test[0]]
clustered_songs_input_cluster

In [23]:
random_song_test = clustered_songs_input_cluster['title'][random.randint(0, len(clustered_songs_input_cluster)-1)]
random_artist_test = clustered_songs_input_cluster.loc[clustered_songs_input_cluster.title == random_song_test, 'artists'].values[0]

In [24]:
random_song_test

In [25]:
random_artist_test

In [26]:
type(random_artist_test)

In [127]:
def getArtists(artist_list):
    #convert string containing list to list
    artist_list = ast.literal_eval(artist_list)
    if len(artist_list) > 1:
        artists = '0'
        for artist in artist_list:
            artists = artists+', '+artist
            artists = artists.replace('0', '')
    else:
        artists = artist_list[0]
    return artists

In [28]:
getArtists(random_artist_test)

#### SongRecommender Function

In [138]:
def checkSongAndRecommend():
    song_input = input("Enter your favorite song: ")
    artist_input = input("Enter the artist: ")
    
    #lowercase song input and hot100songs title column
    song_input_lower = song_input.lower()
    hot100songs['title_lower'] = hot100songs['title'].str.lower()
    
    #Checking if song title is in top100
    if len(hot100songs[hot100songs.title_lower.str.contains(song_input_lower)]) > 0:
        random_song = hot100songs['title'][random.randint(0, len(hot100songs)-1)]
        random_artist = hot100songs.loc[hot100songs.title == random_song, 'artists'].values[0]
        
        return 'Look at this new hot song: '+random_song+' from '+random_artist
    
    #If song is not in top100 list:
    else:
        #Search uri for input song on spotify
        #song_spotify = sp.search(q=song_input, limit=1)
        song_spotify = sp.search(q=song_input+' '+artist_input, limit=1)
        song_uri = song_spotify['tracks']['items'][0]['uri']
        
        #Collect audio features for input song
        song_audio_features = sp.audio_features(song_uri)
        audio_feat_df = pd.DataFrame(song_audio_features)
        audio_feat_df_corr = audio_feat_df[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 
                          'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']].copy()
        
        #Scale audio feature columns
        audio_feat_df_corr_prep = scaler.transform(audio_feat_df_corr)
        
        # Predicting / assigning the clusters:
        clusters = kmeans.predict(audio_feat_df_corr_prep)
        
        #take clustered_songs dataframe and search for a song & artist with the same number as clusters result
        clustered_songs_input_cluster = clustered_songs[clustered_songs['cluster'] == clusters[0]].reset_index()
        random_song_output = clustered_songs_input_cluster['title'][random.randint(0, len(clustered_songs_input_cluster['title'])-1)]
        random_artist_output = clustered_songs_input_cluster.loc[clustered_songs_input_cluster.title == random_song_output, 
                                'artists'].values[0]
        random_artist_output = getArtists(random_artist_output)
        
        return 'Found a similar song: '+random_song_output+' from '+random_artist_output

In [ ]:
#Former checkSongAndRecommend function:
#def checkSongAndRecommend():
#    song_input = input("Enter your favorite song: ")
#    song_input_lower = song_input.lower()
#    #Checking if song title is in top100
#    if song_input_lower in [x.lower() for x in hot100songs['title'].unique()]:
#        random_song = hot100songs['title'][random.randint(0, len(hot100songs)-1)]
#        random_artist = hot100songs.loc[hot100songs.title == random_song, 'artists'].values[0]
#        return 'Look at this new hot song: '+random_song+' from '+random_artist
    #If song is not in top100 list:
#    else:
        #Search uri for input song on spotify
#        song_spotify = sp.search(q=song_input, limit=1)
#        song_uri = song_spotify['tracks']['items'][0]['uri']
        #Collect audio features for input song
 #       song_audio_features = sp.audio_features(song_uri)
 #       audio_feat_df = pd.DataFrame(song_audio_features)
 #       audio_feat_df_corr = audio_feat_df[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 
 #                         'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']].copy()
        #Scale audio feature columns
 #       audio_feat_df_corr_prep = scaler.transform(audio_feat_df_corr)
        # Predicting / assigning the clusters:
 #       clusters = kmeans.predict(audio_feat_df_corr_prep)
        #take clustered_songs dataframe and search for a song with the same number as clusters_test
 #       clustered_songs_input_cluster = clustered_songs[clustered_songs['cluster'] == clusters[0]].reset_index()
        #print('clustered songs: ', clustered_songs_input_cluster)
 #       random_song_output = clustered_songs_input_cluster['title'][random.randint(0, len(clustered_songs_input_cluster['title'])-1)]
        #print('Random song: ', random_song_output)
 #       random_artist_output = clustered_songs_input_cluster.loc[clustered_songs_input_cluster.title == random_song_output, 
#                                'artists'].values[0]
 #       random_artist_output = getArtists(random_artist_output)
 #       return 'Found a similar song: '+random_song_output+' from '+random_artist_output

#### Test the complete function

In [32]:
checkSongAndRecommend()

Enter your favorite song: Flowers


'Look at this new hot song: Alone from Jimin'

In [140]:
checkSongAndRecommend()

Enter your favorite song: toxic
Enter the artist: Britney Spears


'Found a similar song: Frozen - Extended Club Mix Edit from Madonna'